# Django request API on model

In [1]:
# if django shell_plus unavailable
import movies.wsgi

In [2]:
# if error async whith django 3.0+
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
# import classes from model 
from movieweb.models import Movie, Star, Play
from django.db.models import F,Q, Value as V, CharField, Case, When, Min, Max, Avg, Count
from django.db.models.functions import Extract, Length, Substr ,Left, Coalesce, Concat, Upper
from datetime import date

## Simple requests

In [4]:
Movie.objects.all()

<QuerySet [<Movie: Family Plot (1976, #74512)>, <Movie: Frenzy (1972, #68611)>, <Movie: Topaz (1969, #65112)>, <Movie: Torn Curtain (1966, #61107)>, <Movie: Marnie (1964, #58329)>, <Movie: The Birds (1963, #56869)>, <Movie: Psycho (1960, #54215)>, <Movie: North by Northwest (1959, #53125)>, <Movie: Vertigo (1958, #52357)>, <Movie: The Wrong Man (1956, #51207)>, <Movie: The Man Who Knew Too Much (1956, #49470)>, <Movie: The Trouble with Harry (1955, #48750)>, <Movie: To Catch a Thief (1955, #48728)>, <Movie: Rear Window (1954, #47396)>, <Movie: Dial M for Murder (1954, #46912)>, <Movie: I Confess (1953, #45897)>, <Movie: Strangers on a Train (1951, #44079)>, <Movie: Stage Fright (1950, #42994)>, <Movie: Under Capricorn (1949, #42004)>, <Movie: Rope (1948, #40746)>, '...(remaining elements truncated)...']>

In [5]:
Star.objects.all()

<QuerySet [<Star: Star object (2588606)>, <Star: Star object (33)>, <Star: Star object (54)>, <Star: Star object (78)>, <Star: Star object (116)>, <Star: Star object (125)>, <Star: Star object (142)>, <Star: Star object (148)>, <Star: Star object (216)>, <Star: Star object (230)>, <Star: Star object (233)>, <Star: Star object (537)>, <Star: Star object (1230)>, <Star: Star object (947)>, <Star: Star object (1136)>, <Star: Star object (364455)>, <Star: Star object (1137)>, <Star: Star object (491590)>, <Star: Star object (626350)>, <Star: Star object (1340)>, '...(remaining elements truncated)...']>

In [7]:
q = Star.objects.all()
print(q.query)

SELECT "stars"."id", "stars"."name", "stars"."birthdate" FROM "stars"


In [9]:
class Toto:
    pass
t = Toto()
print(str(t))
print(repr(t))
print(type(t))

<class '__main__.Toto'>


In [11]:
m = Movie.objects.get(pk=74512)
m

<Movie: Family Plot (1976, #74512)>

In [12]:
# associated object Star, foreign key id_director
m.director, m.director_id

(<Star: Alfred Hitchcock (1899-08-13, #33)>, 33)

In [4]:
Star.objects.get(pk=1)

<Star: Fred Astaire (1899-01-01 #1)>

In [5]:
Star.objects.filter(name__endswith='McQueen')

<QuerySet [<Star: Steve McQueen (1969-10-09 #2588606)>, <Star: Steve McQueen (1930-03-24 #537)>, <Star: Vernon 'Red' McQueen (None #574371)>, <Star: Butterfly McQueen (1911-01-01 #574335)>, <Star: Chad McQueen (1960-01-01 #574337)>, <Star: Ron McQueen (None #574360)>, <Star: Casper McQueen (None #10574857)>, <Star: Marney McQueen (None #1929923)>, <Star: Scott McQueen (None #7290189)>, <Star: Adrienne McQueen (None #1432037)>]>

In [6]:
# selection: filter
# projection: values
Star.objects.filter(name__endswith='McQueen').values('name')

<QuerySet [{'name': 'Steve McQueen'}, {'name': 'Steve McQueen'}, {'name': "Vernon 'Red' McQueen"}, {'name': 'Butterfly McQueen'}, {'name': 'Chad McQueen'}, {'name': 'Ron McQueen'}, {'name': 'Casper McQueen'}, {'name': 'Marney McQueen'}, {'name': 'Scott McQueen'}, {'name': 'Adrienne McQueen'}]>

In [21]:
# https://docs.djangoproject.com/en/3.1/ref/models/database-functions/
# fonction en ligne
# select name, extract(year from birthdate) as year from stars where name like '%McQueen' 
Star.objects.filter(name__endswith='McQueen').values('name', year=Extract('birthdate','year'))

<QuerySet [{'name': 'Steve McQueen', 'year': 1969}, {'name': 'Steve McQueen', 'year': 1930}, {'name': "Vernon 'Red' McQueen", 'year': None}, {'name': 'Butterfly McQueen', 'year': 1911}, {'name': 'Chad McQueen', 'year': 1960}, {'name': 'Ron McQueen', 'year': None}, {'name': 'Casper McQueen', 'year': None}, {'name': 'Marney McQueen', 'year': None}, {'name': 'Scott McQueen', 'year': None}, {'name': 'Adrienne McQueen', 'year': None}]>

In [11]:
Star.objects.filter(birthdate=date(1930,3,24))

<QuerySet [<Star: Steve McQueen (1930-03-24 #537)>]>

In [19]:
# stars nées en 1930
# select * where extract(year from birthdate) = 1930
q = Star.objects.annotate(year=Extract('birthdate','year')).filter(year=1930)
print(q.query)
q

SELECT "stars"."id", "stars"."name", "stars"."birthdate", EXTRACT('year' FROM "stars"."birthdate") AS "year" FROM "stars" WHERE EXTRACT('year' FROM "stars"."birthdate") = 1930


<QuerySet [<Star: Sean Connery (1930-08-25 #125)>, <Star: Clint Eastwood (1930-05-31 #142)>, <Star: Steve McQueen (1930-03-24 #537)>, <Star: Philippe Noiret (1930-01-01 #634159)>, <Star: Abel Fernandez (1930-01-01 #273359)>, <Star: Carolyn Conwell (1930-01-01 #176804)>, <Star: Nancy Kilgas (1930-01-01 #453025)>, <Star: Norbert Meisel (1930-01-01 #577025)>, <Star: Tippi Hedren (1930-01-01 #1335)>, <Star: Rod Taylor (1930-01-01 #1792)>, <Star: Roxanne Tunis (1930-01-01 #876651)>, <Star: Lola D'Annunzio (1930-01-01 #195340)>, <Star: Dan Terranova (1930-01-01 #855763)>, <Star: Carolyn Jones (1930-01-01 #427700)>, <Star: Petra Davies (1930-01-01 #203904)>, <Star: Henry Blair (1930-01-01 #86259)>, <Star: Lawrence Osman (1930-01-01 #652058)>, <Star: Raymond Severn (1930-01-01 #786358)>, <Star: Raymond St. Jacques (1930-01-01 #820566)>, <Star: Chester Smith (1930-01-01 #1143592)>, '...(remaining elements truncated)...']>

In [34]:
# order by:     .order_by(Length('title').desc()) \
Movie.objects.annotate(title_length=Length('title')) \
    .filter(title_length__gt=100) \
    .order_by(F('title_length').desc()) \
    .values('title', 'title_length')

<QuerySet [{'title': 'Night of the Day of the Dawn of the Son of the Bride of the Return of the Revenge of the Terror of the Attack of the Evil Mutant Hellbound Flesh Eating Crawling Alien Zombified Subhumanoid Living Dead, Part 5', 'title_length': 208}, {'title': 'Night of the Day of the Dawn of the Son of the Bride of the Return of the Revenge of the Terror of the Attack of the Evil, Mutant, Hellbound, Flesh-Eating Subhumanoid Zombified Living Dead, Part 3', 'title_length': 196}]>

In [39]:
# silly request comparing two columns
Movie.objects.filter(year__gt=F('duration'))

<QuerySet [<Movie: Family Plot (1976 #74512)>, <Movie: Frenzy (1972 #68611)>, <Movie: Topaz (1969 #65112)>, <Movie: Torn Curtain (1966 #61107)>, <Movie: Marnie (1964 #58329)>, <Movie: The Birds (1963 #56869)>, <Movie: Psycho (1960 #54215)>, <Movie: North by Northwest (1959 #53125)>, <Movie: Vertigo (1958 #52357)>, <Movie: The Wrong Man (1956 #51207)>, <Movie: The Man Who Knew Too Much (1956 #49470)>, <Movie: The Trouble with Harry (1955 #48750)>, <Movie: To Catch a Thief (1955 #48728)>, <Movie: Rear Window (1954 #47396)>, <Movie: Dial M for Murder (1954 #46912)>, <Movie: I Confess (1953 #45897)>, <Movie: Strangers on a Train (1951 #44079)>, <Movie: Stage Fright (1950 #42994)>, <Movie: Under Capricorn (1949 #42004)>, <Movie: Rope (1948 #40746)>, '...(remaining elements truncated)...']>

In [41]:
# combiner des predicats: &, |, ~
Movie.objects.filter(Q(year__gte=1970) & Q(year__lte=1979))

<QuerySet [<Movie: Family Plot (1976 #74512)>, <Movie: Frenzy (1972 #68611)>, <Movie: Big Jake (1971 #66831)>, <Movie: The Shootist (1976 #75213)>, <Movie: Rooster Cogburn (1975 #73636)>, <Movie: Brannigan (1975 #72732)>, <Movie: McQ (1974 #71824)>, <Movie: Cahill U.S. Marshal (1973 #69834)>, <Movie: The Train Robbers (1973 #70825)>, <Movie: Cancel My Reservation (1972 #68332)>, <Movie: The Cowboys (1972 #68421)>, <Movie: Rio Lobo (1970 #66301)>, <Movie: Chisum (1970 #65547)>, <Movie: Xenogenesis (1978 #251488)>, <Movie: Cuba (1979 #79013)>, <Movie: Meteor (1979 #79550)>, <Movie: The Great Train Robbery (1978 #79240)>, <Movie: A Bridge Too Far (1977 #75784)>, <Movie: Circasia (1976 #6164290)>, <Movie: The Next Man (1976 #74962)>, '...(remaining elements truncated)...']>

In [44]:
Movie.objects.annotate(decade= F('year') / 10).filter(decade=197)

<QuerySet [<Movie: Family Plot (1976 #74512)>, <Movie: Frenzy (1972 #68611)>, <Movie: Big Jake (1971 #66831)>, <Movie: The Shootist (1976 #75213)>, <Movie: Rooster Cogburn (1975 #73636)>, <Movie: Brannigan (1975 #72732)>, <Movie: McQ (1974 #71824)>, <Movie: Cahill U.S. Marshal (1973 #69834)>, <Movie: The Train Robbers (1973 #70825)>, <Movie: Cancel My Reservation (1972 #68332)>, <Movie: The Cowboys (1972 #68421)>, <Movie: Rio Lobo (1970 #66301)>, <Movie: Chisum (1970 #65547)>, <Movie: Xenogenesis (1978 #251488)>, <Movie: Cuba (1979 #79013)>, <Movie: Meteor (1979 #79550)>, <Movie: The Great Train Robbery (1978 #79240)>, <Movie: A Bridge Too Far (1977 #75784)>, <Movie: Circasia (1976 #6164290)>, <Movie: The Next Man (1976 #74962)>, '...(remaining elements truncated)...']>

In [71]:
# produire des codes de movies avec les 3 1eres lettres en majuscule du titre + l'année
# Ex: Mulan de de 2020 => MUL_2020
Movie.objects.values('title', 'year', 
        code=Concat(
                Case(When(Length('title')==1, then=V('___')),
                      When(Length('title')==2, then=V('___')),
                      default=Upper(Left('title',3))),
                V('_'),
                'year', 
                 output_field=CharField()))

TypeError: When() supports a Q object, a boolean expression, or lookups as a condition.

In [72]:
# Aggregate functions : count, min, max, avg
Movie.objects.annotate(decade= F('year') / 10).filter(decade=197).count()


145

In [79]:
# without group by
Movie.objects.annotate(decade= F('year') / 10).filter(decade=197) \
        .aggregate(duration_min=Min('duration'), duration_max=Max('duration'))

{'duration_min': 1, 'duration_max': 175}

In [82]:
# with group by
q = Movie.objects.annotate(decade= F('year') / 10).filter(decade=197) \
        .values('year') \
        .annotate(duration_min=Min('duration'), duration_max=Max('duration'))
print(q.query)
q

SELECT "movies"."year", MIN("movies"."duration") AS "duration_min", MAX("movies"."duration") AS "duration_max" FROM "movies" WHERE ("movies"."year" / 10) = 197 GROUP BY "movies"."year"


<QuerySet [{'year': 1970, 'duration_min': 71, 'duration_max': 144}, {'year': 1971, 'duration_min': 12, 'duration_max': 120}, {'year': 1972, 'duration_min': 2, 'duration_max': 134}, {'year': 1973, 'duration_min': 1, 'duration_max': 151}, {'year': 1974, 'duration_min': 49, 'duration_max': 165}, {'year': 1975, 'duration_min': 80, 'duration_max': 129}, {'year': 1976, 'duration_min': 11, 'duration_max': 135}, {'year': 1977, 'duration_min': 70, 'duration_max': 175}, {'year': 1978, 'duration_min': 7, 'duration_max': 145}, {'year': 1979, 'duration_min': 1, 'duration_max': 147}]>

## Queries following assocations

In [13]:
Movie.objects.filter(director__name='Steve McQueen')

<QuerySet [<Movie: Widows (2018, #4218572)>, <Movie: Bleu de Chanel: The Film (2018, #11911992)>, <Movie: Mr. Burberry (2016, #5620080)>, <Movie: Kanye West: All Day/I Feel Like That (2015, #4510600)>, <Movie: Ashes (2014, #4149768)>, <Movie: 12 Years a Slave (2013, #2024544)>, <Movie: Shame (2011, #1723811)>, <Movie: Giardini (2009, #3329434)>, <Movie: Static (2009, #2552452)>, <Movie: Rayners Lane (2008, #3327650)>, <Movie: Hunger (2008, #986233)>, <Movie: Gravesend (2007, #3329376)>, <Movie: Running Thunder (2007, #3329378)>, <Movie: Unexploded (2007, #3329392)>, <Movie: Pursuit (2005, #3327696)>, <Movie: Charlotte (2004, #3327694)>, <Movie: Carib's Leap (2002, #3327702)>, <Movie: Western Deep (2002, #3323988)>, <Movie: 7th Nov. (2001, #3327668)>, <Movie: Girls, Tricky (2001, #3327674)>, '...(remaining elements truncated)...']>

In [14]:
Movie.objects.filter(director__name__endswith='McQueen')

<QuerySet [<Movie: Widows (2018, #4218572)>, <Movie: Bleu de Chanel: The Film (2018, #11911992)>, <Movie: Mr. Burberry (2016, #5620080)>, <Movie: Kanye West: All Day/I Feel Like That (2015, #4510600)>, <Movie: Ashes (2014, #4149768)>, <Movie: 12 Years a Slave (2013, #2024544)>, <Movie: Shame (2011, #1723811)>, <Movie: Giardini (2009, #3329434)>, <Movie: Static (2009, #2552452)>, <Movie: Rayners Lane (2008, #3327650)>, <Movie: Hunger (2008, #986233)>, <Movie: Gravesend (2007, #3329376)>, <Movie: Running Thunder (2007, #3329378)>, <Movie: Unexploded (2007, #3329392)>, <Movie: Pursuit (2005, #3327696)>, <Movie: Charlotte (2004, #3327694)>, <Movie: Carib's Leap (2002, #3327702)>, <Movie: Western Deep (2002, #3323988)>, <Movie: 7th Nov. (2001, #3327668)>, <Movie: Girls, Tricky (2001, #3327674)>, '...(remaining elements truncated)...']>

In [33]:
# stats par realisateur: nb de films, année du 1er film, année du dernier film
# nb: exclude force inner join instead of left outer join
threashold = 30
q = Movie.objects \
        .exclude(director=None) \
        .values('director', 'director__name') \
        .annotate(nb_movies=Count('id'), first_year=Min('year'), last_year=Max('year')) \
        .filter(nb_movies__gt=threashold) \
        .order_by(F('nb_movies').desc())
print(q.query)
q
        

SELECT "movies"."id_director", "stars"."name", COUNT("movies"."id") AS "nb_movies", MIN("movies"."year") AS "first_year", MAX("movies"."year") AS "last_year" FROM "movies" INNER JOIN "stars" ON ("movies"."id_director" = "stars"."id") WHERE NOT ("movies"."id_director" IS NULL) GROUP BY "movies"."id_director", "stars"."name" HAVING COUNT("movies"."id") > 30 ORDER BY COUNT("movies"."id") DESC


<QuerySet [{'director': 33, 'director__name': 'Alfred Hitchcock', 'nb_movies': 60, 'first_year': 1922, 'last_year': 1976}, {'director': 217, 'director__name': 'Martin Scorsese', 'nb_movies': 48, 'first_year': 1959, 'last_year': 2019}, {'director': 229, 'director__name': 'Steven Spielberg', 'nb_movies': 41, 'first_year': 1959, 'last_year': 2018}, {'director': 142, 'director__name': 'Clint Eastwood', 'nb_movies': 39, 'first_year': 1971, 'last_year': 2019}, {'director': 361, 'director__name': 'Brian De Palma', 'nb_movies': 38, 'first_year': 1960, 'last_year': 2019}, {'director': 631, 'director__name': 'Ridley Scott', 'nb_movies': 35, 'first_year': 1965, 'last_year': 2019}, {'director': 247, 'director__name': 'John Woo', 'nb_movies': 34, 'first_year': 1968, 'last_year': 2017}, {'director': 2588606, 'director__name': 'Steve McQueen', 'nb_movies': 31, 'first_year': 1993, 'last_year': 2018}, {'director': 600, 'director__name': 'Sam Raimi', 'nb_movies': 31, 'first_year': 1972, 'last_year': 201

In [38]:
Movie.objects.filter(director__name__iregex=r'(^| )queen($| )')

<QuerySet []>

In [23]:
# liste des ids de réalisateur
Movie.objects.values('director').distinct().order_by('director')

<QuerySet [{'director': 33}, {'director': 41}, {'director': 78}, {'director': 95}, {'director': 116}, {'director': 125}, {'director': 142}, {'director': 158}, {'director': 165}, {'director': 184}, {'director': 204}, {'director': 212}, {'director': 217}, {'director': 229}, {'director': 230}, {'director': 231}, {'director': 233}, {'director': 241}, {'director': 247}, {'director': 265}, '...(remaining elements truncated)...']>

In [44]:
# prefetch each director by joining table Star
movies = Movie.objects.select_related('director')
print(movies.query)
# directors as Star objects are already here
directors = [m.director for m in movies ]
directors

SELECT "movies"."id", "movies"."title", "movies"."year", "movies"."duration", "movies"."id_director", "stars"."id", "stars"."name", "stars"."birthdate" FROM "movies" LEFT OUTER JOIN "stars" ON ("movies"."id_director" = "stars"."id")


[<Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Al

In [43]:
movies = Movie.objects.all()
print(movies.query)
# fetch director in database for each movie
directors = [m.director for m in movies ]
directors

SELECT "movies"."id", "movies"."title", "movies"."year", "movies"."duration", "movies"."id_director" FROM "movies"


[<Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Alfred Hitchcock (1899-08-13, #33)>,
 <Star: Al

In [5]:
s = Star.objects.get(pk=33)
s.directedMovies.all()

<QuerySet [<Movie: Family Plot (1976, #74512)>, <Movie: Frenzy (1972, #68611)>, <Movie: Topaz (1969, #65112)>, <Movie: Torn Curtain (1966, #61107)>, <Movie: Marnie (1964, #58329)>, <Movie: The Birds (1963, #56869)>, <Movie: Psycho (1960, #54215)>, <Movie: North by Northwest (1959, #53125)>, <Movie: Vertigo (1958, #52357)>, <Movie: The Wrong Man (1956, #51207)>, <Movie: The Man Who Knew Too Much (1956, #49470)>, <Movie: The Trouble with Harry (1955, #48750)>, <Movie: To Catch a Thief (1955, #48728)>, <Movie: Rear Window (1954, #47396)>, <Movie: Dial M for Murder (1954, #46912)>, <Movie: I Confess (1953, #45897)>, <Movie: Strangers on a Train (1951, #44079)>, <Movie: Stage Fright (1950, #42994)>, <Movie: Under Capricorn (1949, #42004)>, <Movie: Rope (1948, #40746)>, '...(remaining elements truncated)...']>

In [6]:
# directors of movies from year 2020
# reverse following association
q = Star.objects.filter(playedMovies__year=2020)
print(q.query)
q

SELECT "stars"."id", "stars"."name", "stars"."birthdate" FROM "stars" INNER JOIN "play3" ON ("stars"."id" = "play3"."id_actor") INNER JOIN "movies" ON ("play3"."id_movie" = "movies"."id") WHERE "movies"."year" = 2020


<QuerySet [<Star: Harrison Ford (1942-07-13, #148)>, <Star: Omar Sy (1978-01-01, #1082477)>, <Star: Cara Gee (None, #4446254)>, <Star: Dan Stevens (1982-01-01, #1405398)>, <Star: Bradley Whitford (1959-01-01, #925966)>, <Star: Jean Louisa Kelly (1972-01-01, #446465)>, <Star: Michael Horse (1949-01-01, #395398)>, <Star: Karen Gillan (1987-01-01, #2394794)>, <Star: Colin Woodell (1991-01-01, #6018521)>, <Star: Micah Fitzgerald (None, #4288087)>, <Star: Heather McPhaul (1962-01-01, #1573698)>, <Star: Adam Fergus (None, #1727237)>, <Star: Stephanie Czajkowski (1973-01-01, #1570123)>, <Star: Abraham Benrubi (1969-01-01, #72344)>, <Star: Thomas Adoue Polk (None, #6930836)>, <Star: Raven Scott (None, #7258294)>, <Star: Brad Greenquist (1959-01-01, #339159)>, <Star: Benjamin Hoffman (None, #1887263)>, <Star: Alexander Schoenauer (None, #11374318)>, <Star: Aria Lyric Leabu (2008-01-01, #6507096)>, '...(remaining elements truncated)...']>

In [7]:
# query on directed movies
s = Star.objects.get(pk=33)
moviesHitchcock = s.directedMovies.all().order_by('year')
moviesHitchcock

<QuerySet [<Movie: Number 13 (1922, #13444)>, <Movie: The Pleasure Garden (1925, #16230)>, <Movie: The Mountain Eagle (1926, #16127)>, <Movie: The Ring (1927, #18328)>, <Movie: The Lodger: A Story of the London Fog (1927, #17075)>, <Movie: When Boys Leave Home (1927, #17825)>, <Movie: Easy Virtue (1927, #17843)>, <Movie: The Farmer's Wife (1928, #18876)>, <Movie: Champagne (1928, #18756)>, <Movie: Sound Test for Blackmail (1929, #249159)>, <Movie: The Manxman (1929, #20142)>, <Movie: Blackmail (1929, #19702)>, <Movie: Murder! (1930, #21165)>, <Movie: Juno and the Paycock (1930, #21015)>, <Movie: An Elastic Affair (1930, #295228)>, <Movie: East of Shanghai (1931, #23395)>, <Movie: Mary (1931, #21128)>, <Movie: The Skin Game (1931, #22395)>, <Movie: Number 17 (1932, #23285)>, <Movie: The Man Who Knew Too Much (1934, #25452)>, '...(remaining elements truncated)...']>

In [11]:
plays = Play.objects.filter(role='James Bond').order_by('movie__year')
for p in plays:
    print(p.movie.year, p.actor.name, p.movie.title, p.role, sep=', ')


1962, Sean Connery, Dr. No, James Bond
1963, Sean Connery, From Russia with Love, James Bond
1964, Sean Connery, Goldfinger, James Bond
1965, Sean Connery, Thunderball, James Bond
1965, Frank Cousins, Thunderball, James Bond
1967, Sean Connery, You Only Live Twice, James Bond
1969, George Lazenby, On Her Majesty's Secret Service, James Bond
1971, Sean Connery, Diamonds Are Forever, James Bond
1973, Roger Moore, Live and Let Die, James Bond
1974, Roger Moore, The Man with the Golden Gun, James Bond
1977, Roger Moore, The Spy Who Loved Me, James Bond
1979, Roger Moore, Moonraker, James Bond
1983, Sean Connery, Never Say Never Again, James Bond
1983, Roger Moore, Octopussy, James Bond
1985, Roger Moore, A View to a Kill, James Bond
1987, Timothy Dalton, The Living Daylights, James Bond
1989, Timothy Dalton, Licence to Kill, James Bond
1995, Pierce Brosnan, GoldenEye, James Bond
1997, Pierce Brosnan, Tomorrow Never Dies, James Bond
1999, Pierce Brosnan, The World Is Not Enough, James Bond


In [14]:
movies007 = Movie.objects.filter(play__role='James Bond').order_by('year')
movies007

<QuerySet [<Movie: Dr. No (1962, #55928)>, <Movie: From Russia with Love (1963, #57076)>, <Movie: Goldfinger (1964, #58150)>, <Movie: Thunderball (1965, #59800)>, <Movie: Thunderball (1965, #59800)>, <Movie: You Only Live Twice (1967, #62512)>, <Movie: On Her Majesty's Secret Service (1969, #64757)>, <Movie: Diamonds Are Forever (1971, #66995)>, <Movie: Live and Let Die (1973, #70328)>, <Movie: The Man with the Golden Gun (1974, #71807)>, <Movie: The Spy Who Loved Me (1977, #76752)>, <Movie: Moonraker (1979, #79574)>, <Movie: Never Say Never Again (1983, #86006)>, <Movie: Octopussy (1983, #86034)>, <Movie: A View to a Kill (1985, #90264)>, <Movie: The Living Daylights (1987, #93428)>, <Movie: Licence to Kill (1989, #97742)>, <Movie: GoldenEye (1995, #113189)>, <Movie: Tomorrow Never Dies (1997, #120347)>, <Movie: The World Is Not Enough (1999, #143145)>, '...(remaining elements truncated)...']>

In [15]:
m = movies007[0]
m.director


<Star: Terence Young (1915-01-01, #950109)>

In [17]:
m.actors.all()

<QuerySet [<Star: Sean Connery (1930-08-25, #125)>, <Star: Ursula Andress (1936-01-01, #266)>, <Star: Joseph Wiseman (1918-01-01, #936476)>, <Star: Jack Lord (1920-01-01, #520437)>, <Star: Bernard Lee (1908-01-01, #496866)>, <Star: Anthony Dawson (1916-01-01, #206060)>, <Star: Zena Marshall (1925-01-01, #551243)>, <Star: John Kitzmiller (1913-01-01, #457839)>, <Star: Eunice Gayson (1928-01-01, #311013)>, <Star: Lois Maxwell (1927-01-01, #561755)>, <Star: Peter Burton (1921-01-01, #78252)>, <Star: Yvonne Shima (None, #793553)>, <Star: Michel Mok (None, #596304)>, <Star: Marguerite LeWars (1938-01-01, #506759)>, <Star: William Foster-Davis (1904-01-01, #288126)>, <Star: Dolores Keator (1925-01-01, #444180)>, <Star: Reggie Carter (1936-01-01, #141883)>, <Star: Louis Blaazer (1925-01-01, #85108)>, <Star: Colonel Burton (None, #123552)>, <Star: Abbot Anderson (None, #6088119)>, '...(remaining elements truncated)...']>

In [18]:
Movie.objects.filter(actors__birthdate__year=1930)

<QuerySet [<Movie: Topaz (1969, #65112)>, <Movie: Topaz (1969, #65112)>, <Movie: Torn Curtain (1966, #61107)>, <Movie: Torn Curtain (1966, #61107)>, <Movie: Torn Curtain (1966, #61107)>, <Movie: Marnie (1964, #58329)>, <Movie: Marnie (1964, #58329)>, <Movie: The Birds (1963, #56869)>, <Movie: The Birds (1963, #56869)>, <Movie: The Birds (1963, #56869)>, <Movie: The Wrong Man (1956, #51207)>, <Movie: The Wrong Man (1956, #51207)>, <Movie: The Man Who Knew Too Much (1956, #49470)>, <Movie: Stage Fright (1950, #42994)>, <Movie: Foreign Correspondent (1940, #32484)>, <Movie: Foreign Correspondent (1940, #32484)>, <Movie: Foreign Correspondent (1940, #32484)>, <Movie: The Green Berets (1968, #63035)>, <Movie: The Alamo (1960, #53580)>, <Movie: Rooster Cogburn (1975, #73636)>, '...(remaining elements truncated)...']>

In [10]:
m

NameError: name 'm' is not defined